<a href="https://colab.research.google.com/github/Heisenberg0203/ML_Home_Credit_RISK/blob/master/Model_with_full_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
!pip install tqdm
import tqdm

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rushikesh0203","key":"a57a54502e5e03998e7362e6218d1695"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:

!kaggle competitions download -c iiitb2020-home-credit-default-risk -p /content

 71% 17.0M/24.0M [00:01<00:00, 13.6MB/s]
100% 24.0M/24.0M [00:01<00:00, 16.6MB/s]
  0% 0.00/380k [00:00<?, ?B/s]
100% 380k/380k [00:00<00:00, 53.2MB/s]
 98% 276M/283M [00:10<00:00, 25.9MB/s]
100% 283M/283M [00:10<00:00, 29.3MB/s]
 83% 97.0M/117M [00:04<00:01, 12.2MB/s]
100% 117M/117M [00:04<00:00, 27.4MB/s] 
 93% 137M/147M [00:07<00:00, 17.4MB/s]
100% 147M/147M [00:07<00:00, 21.5MB/s]
 70% 9.00M/12.9M [00:00<00:00, 13.9MB/s]
100% 12.9M/12.9M [00:00<00:00, 18.8MB/s]
 93% 73.0M/78.9M [00:02<00:00, 24.9MB/s]
100% 78.9M/78.9M [00:02<00:00, 28.3MB/s]
 84% 89.0M/106M [00:02<00:00, 32.3MB/s]
100% 106M/106M [00:02<00:00, 41.4MB/s] 
 88% 41.0M/46.4M [00:01<00:00, 28.6MB/s]
100% 46.4M/46.4M [00:01<00:00, 38.3MB/s]


In [ ]:
!unzip \*.zip

Archive:  application_test.csv.zip
  inflating: application_test.csv    

Archive:  POS_CASH_balance.csv.zip
  inflating: POS_CASH_balance.csv    

Archive:  bureau.csv.zip
  inflating: bureau.csv              

Archive:  application_train.csv.zip
  inflating: application_train.csv   

Archive:  installments_payments.csv.zip
  inflating: installments_payments.csv  

Archive:  bureau_balance.csv.zip
  inflating: bureau_balance.csv      

Archive:  previous_application.csv.zip
  inflating: previous_application.csv  

Archive:  credit_card_balance.csv.zip
  inflating: credit_card_balance.csv  

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

9 archives were successfully processed.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import gc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
print('WRITING PROGRAM TO REDUCE FILE SIZE....')
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#CONVERT AND RETURN NEW FILE

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

print('PROGRAM COMPLETE....')

WRITING PROGRAM TO REDUCE FILE SIZE....
PROGRAM COMPLETE....


In [ ]:
!ls /content/

application_test.csv	     installments_payments.csv
application_test.csv.zip     installments_payments.csv.zip
application_train.csv	     kaggle.json
application_train.csv.zip    POS_CASH_balance.csv
bureau_balance.csv	     POS_CASH_balance.csv.zip
bureau_balance.csv.zip	     previous_application.csv
bureau.csv		     previous_application.csv.zip
bureau.csv.zip		     sample_data
credit_card_balance.csv      sample_submission.csv
credit_card_balance.csv.zip  sample_submission.csv.zip


In [ ]:
application_train = import_data("/content/application_train.csv")

Memory usage of dataframe is 186.05 MB
Memory usage after optimization is: 45.23 MB
Decreased by 75.7%


In [ ]:
application_test=import_data("/content/application_test.csv")

Memory usage of dataframe is 99.36 MB
Memory usage after optimization is: 26.56 MB
Decreased by 73.3%


In [ ]:
bur_bal = import_data("/content/bureau_balance.csv")

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 202.45 MB
Decreased by 67.6%


In [ ]:
bur = import_data("/content/bureau.csv")

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 184.00 MB
Decreased by 17.3%


In [ ]:
credit_bal = import_data("/content/credit_card_balance.csv")

Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 275.28 MB
Decreased by 59.2%


In [ ]:
installment_payments = import_data("/content/installments_payments.csv")

Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 371.61 MB
Decreased by 55.3%


In [ ]:
pos_bal = import_data("/content/POS_CASH_balance.csv")

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 231.40 MB
Decreased by 62.1%


In [ ]:
prev_application = import_data("/content/previous_application.csv")

Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 235.95 MB
Decreased by 50.0%


In [ ]:
gc.enable()
gc.collect()

15

In [ ]:
application_train_org = application_train.copy()

In [ ]:
print("Application Shape: ", application_train.shape)
print("Application Shape: ", application_test.shape)

Application Shape:  (199882, 122)
Application Shape:  (107629, 121)


In [ ]:
def getCnamesNumeric(df):
    return list(df.select_dtypes(exclude=['category','object']).columns)

def getCnamesObject(df):
    return list(df.select_dtypes(include=['category','object']).columns)

def getDetailsAboutMissingValuesAllColumns(df):
  return df.isna().sum()/df.shape[0]

def getDetailsAboutOnlyMissingValuesColumns(df):
  print("Shape: ", df.shape)
  return df[list(df.columns[df.isna().any()])].isna().sum()/df.shape[0]

def dropColumns(df, columnNames):
  for column in columnNames:
    df = df.drop(column, axis = 1)
  return df

def imputeCategoricalMissingValuesUsingMode(df):
  columnName = getCnamesObject(df)
  for col in columnName: df[col].fillna(df[col].mode().values[0], inplace = True)

def imputeNumericMissingValuesUsingMean(df):
  columnName = getCnamesNumeric(df)
  for col in columnName: df[col].fillna(df[col].mean(), inplace = True)

def imputeNumericMissingValuesUsingMode(df):
  columnName = getCnamesNumeric(df)
  for col in columnName: df[col].fillna(df[col].mode()[0], inplace = True)
def imputeNumericMissingValuesUsingMedian(df):
  columnName = getCnamesNumeric(df)
  for col in columnName: df[col].fillna(df[col].median(), inplace = True)



def plotCountPlotForCategoricalFeatures(df, cnamesObject):
  number_of_rows = (len(cnamesObject) + 1)/2
  plt.figure(figsize=(20, 6*number_of_rows))

  for i in range(0,len(cnamesObject)):
    plt.subplot(number_of_rows,2,i+1)
    sns.countplot(y=cnamesObject[i], data = df)
    plt.title(cnamesObject[i])
    plt.tight_layout()

def distributionOfCategoricalFeaturesWRTTarget(df, cnamesObject):
  number_of_rows = (len(cnamesObject) + 1)/2
  plt.figure(figsize=(20, 6*number_of_rows))

  for i in range(0,len(cnamesObject)):
    plt.subplot(number_of_rows,2,i+1)
    sns.countplot(data=df, hue="TARGET", y=cnamesObject[i])
    plt.title(cnamesObject[i])
    plt.tight_layout()

def drawCorrelationMatrix(df) :
  length = len(getCnamesNumeric(df))
  correlaionMatrix = df.corr()
  plt.figure(figsize=(length,length*0.8))
  sns.heatmap(correlaionMatrix, annot=True, cmap = 'viridis')

def drawDistributionPlot(df, cnamesNumeric):
  number_of_rows = (len(cnamesNumeric) + 1)/2
  plt.figure(figsize=(20, 4*number_of_rows))

  for i in range(0, len(cnamesNumeric)):
    plt.subplot(number_of_rows,3,i+1)
    sns.kdeplot(df[cnamesNumeric[i]])
    plt.title(cnamesNumeric[i])
    plt.tight_layout()

def corr_columnwise(df):
  columns = df.columns
  for col in columns:
    at_corr_col = at_corr[col]
    print(col+" +ve")
    print(at_corr_col[at_corr_col>0.9].index.to_list())
    print(col+" -ve")
    print(at_corr_col[at_corr_col<-0.9].index.to_list())

def remove_corr_columns(df,atNC,atCC,df_corr):
  threshold = 0.9
  columns = np.full((df_corr.shape[0],), True, dtype=bool)
  for i in range(df_corr.shape[0]):
      for j in range(i+1, df_corr.shape[0]):
          if df_corr.iloc[i,j] >= threshold:
              if columns[j]:
                  columns[j] = False
  x_temp=df[atNC].columns[columns].to_list()
  return pd.concat([df[x_temp],df[atCC]],axis=1)

def checkOutlier(data, cnames_numeric):
    number_of_rows = (len(cnames_numeric) + 1)/2
    plt.figure(figsize=(20, 4*number_of_rows))
    
    for i in range(0,len(cnames_numeric)):
        plt.subplot(number_of_rows,4,i+1)
        sns.boxplot(x=data[cnames_numeric[i]])
        plt.title(cnames_numeric[i])
        plt.tight_layout()

def removeOutlier(data, cnamesNumeric):
    for i in cnamesNumeric:
        q75, q25 = np.percentile(data.loc[:,i], [75 ,25])
        #Calculate IQR
        iqr = q75 - q25
        #Calculate inner and outer fence
        minimum = q25 - (iqr*1.5)
        maximum = q75 + (iqr*1.5)
        #Replace with NA
        data.loc[data.loc[:,i] < minimum,i] = np.nan
        data.loc[data.loc[:,i] > maximum,i] = np.nan
    imputeNumericMissingValuesUsingMean(data)
    return data

def drawPieChartForCategoricalFeatures(df, columnNames):
  number_of_rows = int((len(columnNames) + 1)/2)
  plt.figure(figsize=(20, 6*number_of_rows))

  for i in range(0,len(columnNames)):
    col = columnNames[i]
    percentage = (df[col].value_counts(dropna=True, normalize=True)*100).to_list()
    labels = list(df[col].unique())
    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(labels, percentage)]
    sizes = list(df[col].value_counts(dropna=True))
    ax1 = plt.subplot(number_of_rows,2,i + 1)
    wedges, autotexts = ax1.pie(sizes, startangle=90)
    #ax1.axis('equal')

    ax1.legend(wedges, labels,
              title=col,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))
    plt.title(col)
plt.show()

def applyOneHotEncoding(df, Columns):
  for col in Columns:
    ohc = OneHotEncoder()
    ohe = ohc.fit_transform(df[col].values.reshape(-1,1)).toarray()
    dfOneHot = pd.DataFrame(ohe, columns=[col+"_"+str(ohc.categories_[0][i])
                                                for i in range(len(ohc.categories_[0]))])
    df = pd.concat([df, dfOneHot.reindex(df.index)], axis=1)
    df = df.drop(col, axis = 1)
  return df
  
def plot_feature_importances(df):
    
    df = df.sort_values('importance', ascending = False).reset_index()
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    ax.barh(list(reversed(list(df.index[:10]))), df['importance_normalized'].head(10))

    ax.set_yticks(list(reversed(list(df.index[:10]))))
    ax.set_yticklabels(df['feature'].head(10))
  
    plt.xlabel('Percentage'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
def makereadyforjoin(df):
  temp = df['SK_ID_CURR']
  df=df[getCnamesNumeric(df)]
  df['SK_ID_CURR']=temp
  df=df.groupby('SK_ID_CURR').agg(['mean','max','sum','var','median'])
  df.reset_index(inplace=True)
  df.columns = pd.Index([e[0] + "_" + e[1].upper() for e in df.columns.tolist()])
  df.rename(columns={'SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
  return df

In [ ]:
prev_application = makereadyforjoin(prev_application)

In [ ]:
credit_bal = makereadyforjoin(credit_bal)

In [ ]:
pos_bal = makereadyforjoin(pos_bal)

In [ ]:
installment_payments = makereadyforjoin(installment_payments)

In [ ]:
application_train = application_train.merge(right=prev_application, how='left', on='SK_ID_CURR')
application_test = application_test.merge(right=prev_application, how='left', on='SK_ID_CURR')

In [ ]:
application_train = application_train.merge(right=credit_bal, how='left', on='SK_ID_CURR')
application_test = application_test.merge(right=credit_bal, how='left', on='SK_ID_CURR')

In [ ]:
application_train = application_train.merge(right=pos_bal, how='left', on='SK_ID_CURR')
application_test = application_test.merge(right=pos_bal, how='left', on='SK_ID_CURR')

In [ ]:
application_train = application_train.merge(right=installment_payments, how='left', on='SK_ID_CURR')
application_test = application_test.merge(right=installment_payments, how='left', on='SK_ID_CURR')

In [ ]:

del application_train['SK_ID_CURR']
del application_test['SK_ID_CURR']
del credit_bal
del pos_bal
del installment_payments
del prev_application
gc.enable()
gc.collect()

0

In [ ]:
application_train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,SK_DPD_MEAN_y,SK_DPD_MAX_y,SK_DPD_SUM_y,SK_DPD_VAR_y,SK_DPD_MEDIAN_y,SK_DPD_DEF_MEAN_y,SK_DPD_DEF_MAX_y,SK_DPD_DEF_SUM_y,SK_DPD_DEF_VAR_y,SK_DPD_DEF_MEDIAN_y,NUM_INSTALMENT_VERSION_MEAN,NUM_INSTALMENT_VERSION_MAX,NUM_INSTALMENT_VERSION_SUM,NUM_INSTALMENT_VERSION_VAR,NUM_INSTALMENT_VERSION_MEDIAN,NUM_INSTALMENT_NUMBER_MEAN,NUM_INSTALMENT_NUMBER_MAX,NUM_INSTALMENT_NUMBER_SUM,NUM_INSTALMENT_NUMBER_VAR,NUM_INSTALMENT_NUMBER_MEDIAN,DAYS_INSTALMENT_MEAN,DAYS_INSTALMENT_MAX,DAYS_INSTALMENT_SUM,DAYS_INSTALMENT_VAR,DAYS_INSTALMENT_MEDIAN,DAYS_ENTRY_PAYMENT_MEAN,DAYS_ENTRY_PAYMENT_MAX,DAYS_ENTRY_PAYMENT_SUM,DAYS_ENTRY_PAYMENT_VAR,DAYS_ENTRY_PAYMENT_MEDIAN,AMT_INSTALMENT_MEAN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_VAR,AMT_INSTALMENT_MEDIAN,AMT_PAYMENT_MEAN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_VAR,AMT_PAYMENT_MEDIAN
0,0,Cash loans,M,N,Y,0,360000.0,1125000.0,33025.5,1125000.0,"Spouse, partner",Commercial associate,Higher education,Married,House / apartment,0.022797,-18943,-7369,-2352.0,-2479,NaN,1,1,0,1,0,0,Managers,2.0,2,2,WEDNESDAY,11,0,0,0,0,1,1,Other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0,22.0,0.000000,1.0,6.045455,12.0,133.0,10.997835,6.0,-2340.00,-2062.0,-51488.0,25776.0,-2352.0,-2352.00,-2080.0,-51744.0,23712.0,-2362.0,5312.074219,6176.430176,116865.632812,9.237629e+05,6132.622559,5312.074219,6176.430176,116865.632812,9.237629e+05,6132.622559
1,0,Cash loans,F,N,Y,0,112500.0,251280.0,13630.5,180000.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.022629,-13894,-1393,-5696.0,-5698,NaN,1,1,0,1,0,0,Sales staff,2.0,2,2,WEDNESDAY,14,0,0,0,0,0,0,Self-employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181763,2.0,10.0,0.188599,0.0,20.109091,46.0,1106.0,210.284175,19.0,-448.75,-9.0,-24688.0,inf,-386.0,-452.75,-9.0,-24896.0,inf,-394.0,15690.834961,410400.000000,862995.937500,3.057419e+09,5925.375000,15690.834961,410400.000000,862995.937500,3.057419e+09,5925.375000
2,0,Cash loans,F,N,Y,0,225000.0,544491.0,15916.5,454500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,0.035797,-19821,-2075,-2120.0,-3379,NaN,1,1,1,1,0,0,Sales staff,2.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,Hotel,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.052734,2.0,40.0,0.105286,1.0,5.342105,11.0,203.0,9.960882,5.0,-429.00,-18.0,-16304.0,inf,-326.5,-440.25,-26.0,-16736.0,inf,-331.0,24780.740234,54872.054688,941668.125000,1.626667e+08,18774.224609,24780.740234,54872.054688,941668.125000,1.626667e+08,18774.224609
3,0,Cash loans,F,N,N,2,211500.0,900000.0,26316.0,900000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.006207,-13034,-2205,-1193.0,-4141,NaN,1,1,1,1,0,0,Laborers,4.0,2,2,THURSDAY,17,0,0,0,0,0,0,Industry: type 3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.250000,2.0,5.0,0.250000,1.0,2.500000,4.0,10.0,1.666667,2.5,-409.00,-364.0,-1636.0,1500.0,-409.0,-415.50,-381.0,-1662.0,1172.0,-411.0,113274.515625,365708.656250,453098.062500,2.832133e+10,29129.804688,113274.515625,365708.656250,453098.062500,2.832133e+10,29129.804688
4,1,Cash loans,F,N,Y,0,90000.0,113760.0,8406.0,90000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.020706,-18393,-1694,-8280.0,-1890,NaN,1,1,1,1,0,0,Sales staff,2.0,3,3,SATURD

In [ ]:
application_train.describe()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,...,SK_DPD_MEAN_y,SK_DPD_MAX_y,SK_DPD_SUM_y,SK_DPD_VAR_y,SK_DPD_MEDIAN_y,SK_DPD_DEF_MEAN_y,SK_DPD_DEF_MAX_y,SK_DPD_DEF_SUM_y,SK_DPD_DEF_VAR_y,SK_DPD_DEF_MEDIAN_y,NUM_INSTALMENT_VERSION_MEAN,NUM_INSTALMENT_VERSION_MAX,NUM_INSTALMENT_VERSION_SUM,NUM_INSTALMENT_VERSION_VAR,NUM_INSTALMENT_VERSION_MEDIAN,NUM_INSTALMENT_NUMBER_MEAN,NUM_INSTALMENT_NUMBER_MAX,NUM_INSTALMENT_NUMBER_SUM,NUM_INSTALMENT_NUMBER_VAR,NUM_INSTALMENT_NUMBER_MEDIAN,DAYS_INSTALMENT_MEAN,DAYS_INSTALMENT_MAX,DAYS_INSTALMENT_SUM,DAYS_INSTALMENT_VAR,DAYS_INSTALMENT_MEDIAN,DAYS_ENTRY_PAYMENT_MEAN,DAYS_ENTRY_PAYMENT_MAX,DAYS_ENTRY_PAYMENT_SUM,DAYS_ENTRY_PAYMENT_VAR,DAYS_ENTRY_PAYMENT_MEDIAN,AMT_INSTALMENT_MEAN,AMT_INSTALMENT_MAX,AMT_INSTALMENT_SUM,AMT_INSTALMENT_VAR,AMT_INSTALMENT_MEDIAN,AMT_PAYMENT_MEAN,AMT_PAYMENT_MAX,AMT_PAYMENT_SUM,AMT_PAYMENT_VAR,AMT_PAYMENT_MEDIAN
count,199882.000000,199882.000000,1.998820e+05,1.998820e+05,199872.000000,1.996970e+05,199882.000000,199882.000000,199882.000000,199882.0,199882.000000,68068.0,199882.0,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,199881.0,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,199882.000000,87079.000000,1.994560e+05,160226.000000,98267.000000,82771.000000,102235.000000,66816.000000,60065.000000,93168.000000,99131.000000,100256.000000,64099.000000,...,188111.000000,188111.000000,188111.000000,1.878810e+05,188111.000000,188111.000000,188111.000000,188111.000000,1.878810e+05,188111.000000,189576.000000,189576.0,189576.0,189123.000000,189576.0,189576.000000,189576.000000,189576.000000,189123.000000,189576.000000,189576.00,189576.0,189576.0,189123.0,189576.0,189570.0,189570.0,189576.0,189119.0,189570.0,1.895760e+05,1.895760e+05,1.895760e+05,1.891230e+05,1.895760e+05,1.895700e+05,1.895700e+05,1.895760e+05,1.891190e+05,1.895700e+05
mean,0.081203,0.419447,1.687525e+05,5.996481e+05,27144.953125,5.389723e+05,0.020920,-16026.845549,63590.955214,NaN,-2992.801198,NaN,1.0,0.820514,0.199323,0.998124,0.281121,0.056518,NaN,2.051185,2.030393,12.057679,0.015149,0.051165,0.040914,0.078531,0.231006,0.179491,0.502930,NaN,NaN,0.117493,0.088684,NaN,0.752441,0.044708,0.078857,0.149780,0.226318,0.232300,...,4.468214,15.786828,357.300955,2.273129e+03,3.267930,0.226992,1.490466,18.271324,9.516869e+01,0.145212,NaN,NaN,NaN,NaN,NaN,9.759363,24.389654,743.529962,123.473948,8.645984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.837561e+04,1.358354e+05,6.708581e+05,2.607405e+09,1.279367e+04,1.872921e+04,1.376612e+05,6.769496e+05,2.954894e+09,1.253979e+04
std,0.273147,0.726197,1.066232e+05,4.025896e+05,14584.699219,3.697378e+05,0.013863,4363.761714,141083.570901,NaN,1507.620138,0.0,0.0,0.383760,0.399492,0.043274,0.449547,0.230921,0.0,0.509039,0.502934,3.265905,0.122145,0.220335,0.198092,0.269007,0.421477,0.383764,0.211182,0.000000e+00,0.000000,0.107483,0.082520,0.000000,0.112732,0.076172,0.134521,0.099976,0.144287,0.161255,...,61.164153,154.441587,5743.395517,3.485424e+04,57.937556,13.707611,31.672685,1445.190020,8.383800e+03,13.196299,0.000000,0.0,NaN,NaN,0.0,11.266113,30.056803,1868.304122,346.779968,10.255763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.313361e+04,2.449020e+05,8.773048e+05,1.690077e+10,1.715081e+04,2.489991e+04,2.471615e+05,9.129189e+05,1.779461e+10,1.779384e+04
min,0.00000

In [ ]:
application_train=application_train.replace(np.inf, np.nan)
application_train=application_train.replace(-np.inf, np.nan)

In [ ]:
application_test=application_test.replace(np.inf, np.nan)
application_test=application_test.replace(-np.inf, np.nan)

In [ ]:
application_train_final = application_train.copy()
application_test_final = application_test.copy()

In [ ]:
application_train = application_train_final.copy()
application_test = application_test_final.copy()

In [ ]:
atCC = getCnamesObject(application_train)
print("Number of categorical columns in application_train: ", len(atCC), "\n", atCC)

atNC = getCnamesNumeric(application_train)
print("Number of numerical columns in application_train: ", len(atNC), "\n", atNC)

atestCC = getCnamesObject(application_test)
print("Number of categorical columns in application_test: ", len(atestCC), "\n", atestCC)

atestNC = getCnamesNumeric(application_test)
print("Number of numerical columns in application_test: ", len(atestNC), "\n", atestNC)

Number of categorical columns in application_train:  16 
 ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
Number of numerical columns in application_train:  355 
 ['TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY'

In [ ]:
# atCC.remove('SK_ID_CURR')
# atestCC.remove('SK_ID_CURR')

In [ ]:
at_corr=application_train.corr()

In [ ]:
new_application_train = remove_corr_columns(application_train,atNC,atCC,at_corr)

In [ ]:
labels = new_application_train['TARGET']

new_application_train, new_application_test = new_application_train.align(application_test, join = 'inner', axis = 1)

new_application_train['TARGET'] = labels

print('Training Features shape: ', new_application_train.shape)
print('Testing Features shape: ', new_application_test.shape)

Training Features shape:  (199882, 224)
Testing Features shape:  (107629, 223)


In [ ]:
# print(getDetailsAboutOnlyMissingValuesColumns(new_application_train).sort_values(ascending=False))
# print(getDetailsAboutOnlyMissingValuesColumns(new_application_test).sort_values(ascending=False))

In [ ]:
imputeCategoricalMissingValuesUsingMode(new_application_train)
imputeNumericMissingValuesUsingMedian(new_application_train)

imputeCategoricalMissingValuesUsingMode(new_application_test)
imputeNumericMissingValuesUsingMedian(new_application_test)

In [ ]:
# checkOutlier(new_application_train, atNC)

In [ ]:
#Extract out target variable
dfTarget = new_application_train.TARGET
new_application_train = new_application_train.drop('TARGET', axis = 1)

In [ ]:
atNC = getCnamesNumeric(new_application_train)
atCC = getCnamesObject(new_application_train)

In [ ]:
# new_application_train = removeOutlier(new_application_train, atNC)

#######################Exploring Categorical Variable########################

In [ ]:
new_application_train.select_dtypes(['object','category']).apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [ ]:
frequency_encoded_variables = ['OCCUPATION_TYPE','ORGANIZATION_TYPE']

In [ ]:
def frequency_encoding(variable):
    t = pd.concat([new_application_train[variable], new_application_test[variable]]).value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[variable] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

In [ ]:
from tqdm import tqdm

In [ ]:
for variable in tqdm(frequency_encoded_variables):
    freq_enc_dict = frequency_encoding(variable)
    new_application_train[variable] = new_application_train[variable].map(lambda x: freq_enc_dict.get(x, np.nan))
    new_application_test[variable] = new_application_test[variable].map(lambda x: freq_enc_dict.get(x, np.nan))

100%|██████████| 2/2 [00:00<00:00, 60.21it/s]


In [ ]:
for col in frequency_encoded_variables:
  new_application_train[col] = new_application_train[col].astype('int32')
  new_application_test[col] = new_application_test[col].astype('int32')

In [ ]:
atNC = getCnamesNumeric(new_application_train)
atCC = getCnamesObject(new_application_train)

In [ ]:
new_application_train=pd.get_dummies(new_application_train)
new_application_test = pd.get_dummies(new_application_test)

In [ ]:
new_application_train, new_application_test = new_application_train.align(new_application_test, join = 'inner', axis = 1)

print('Training Features shape: ', new_application_train.shape)
print('Testing Features shape: ', new_application_test.shape)

Training Features shape:  (199882, 272)
Testing Features shape:  (107629, 272)


In [ ]:
train = new_application_train.copy()
test = new_application_test.copy()
train_labels = labels.copy()
features = list(train.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train)
train = sc.transform(train)
test = sc.transform(test)

In [ ]:
train = pd.DataFrame(train,columns=new_application_train.columns)
test =pd.DataFrame(test,columns=new_application_test.columns)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
labels[labels==0].shape,labels[labels==1].shape

((183651,), (16231,))

In [ ]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features,labels):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        # model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary',
        #                            class_weight = 'balanced', learning_rate = 0.03,scale_pos_weight=9,
        #                            reg_alpha = 0.01, reg_lambda = 0.01, n_jobs = -1, random_state = 50)
        model = lgb.LGBMClassifier(n_estimators=10000,learning_rate=0.03,num_leaves=511,colsample_bytree=0.9,
            subsample=0.8,max_depth=-1,reg_alpha=.1,reg_lambda=.1,min_split_gain=.01,min_child_weight=300,
            class_weight='balanced', boosting_type = 'dart',bagging_fraction = 0.6, bagging_freq = 100
            )
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'],
                  early_stopping_rounds = 200, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
train['TARGET'] = pd.Series(labels)
train['SK_ID_CURR'] = import_data('/content/application_train.csv').SK_ID_CURR
test['SK_ID_CURR'] = import_data('/content/application_test.csv').SK_ID_CURR

Memory usage of dataframe is 186.05 MB
Memory usage after optimization is: 45.23 MB
Decreased by 75.7%
Memory usage of dataframe is 99.36 MB
Memory usage after optimization is: 26.56 MB
Decreased by 73.3%


In [ ]:
submission, feature_importances, metrics = model(train, test)

Training Data Shape:  (199882, 272)
Testing Data Shape:  (107629, 272)
[200]	train's auc: 0.780414	train's binary_logloss: 0.583786	valid's auc: 0.755969	valid's binary_logloss: 0.580931
[400]	train's auc: 0.803117	train's binary_logloss: 0.553508	valid's auc: 0.76583	valid's binary_logloss: 0.557846
[600]	train's auc: 0.818165	train's binary_logloss: 0.537434	valid's auc: 0.769239	valid's binary_logloss: 0.544254
[800]	train's auc: 0.829593	train's binary_logloss: 0.524886	valid's auc: 0.770165	valid's binary_logloss: 0.535015
[1000]	train's auc: 0.840775	train's binary_logloss: 0.512217	valid's auc: 0.771738	valid's binary_logloss: 0.532757
[1200]	train's auc: 0.850132	train's binary_logloss: 0.502999	valid's auc: 0.773055	valid's binary_logloss: 0.518711
[1400]	train's auc: 0.85997	train's binary_logloss: 0.491628	valid's auc: 0.7728	valid's binary_logloss: 0.512067
[1600]	train's auc: 0.866362	train's binary_logloss: 0.483057	valid's auc: 0.773001	valid's binary_logloss: 0.505364
[

In [ ]:
metrics

,fold,train,valid
0,0,0.788964,0.741400
1,1,0.788135,0.746738
2,2,0.788672,0.747869
3,3,0.788854,0.744437
4,4,0.788748,0.745161
5,5,0.789138,0.736319
6,6,0.788981,0.744723
7,7,0.789152,0.735812
8,8,0.788854,0.745586
9,9,0.789293,0.739942


In [ ]:
submission.to_csv('Elite11_lgbm.csv', index = False)

In [ ]:
submission.head()

,SK_ID_CURR,TARGET
0,7auxod,0.045308
1,xur7ur,0.199900
2,eau6rx,0.101285
3,obdr67,0.053920
4,7xbu7d,0.067029


In [ ]:
one = pd.read_csv("Elite8_lgbm.csv")

In [ ]:
two =  pd.read_csv("Elite9_lgbm.csv")

In [ ]:
three = pd.read_csv("Elite10_lgbm.csv")

In [ ]:
one["TARGET2"] = two["TARGET"]

In [ ]:
one["TARGET3"] = three["TARGET"]

In [ ]:
one

,SK_ID_CURR,TARGET,TARGET2,TARGET3
0,7auxod,0.044102,0.043694,0.045308
1,xur7ur,0.203215,0.205553,0.199900
2,eau6rx,0.105135,0.102717,0.101285
3,obdr67,0.054243,0.053841,0.053920
4,7xbu7d,0.062540,0.062075,0.067029
...,...,...,...,...
107624,x7o6ud,0.127500,0.128519,0.132594
107625,ueexer,0.022331,0.022330,0.019634
107626,ou7exx,0.050077,0.051195,0.050738
107627,6ux6rd,0.090136,0.089816,0.088201


In [ ]:
one['TARGET_AVG'] = (one['TARGET'] + one['TARGET2']+one['TARGET3'])/3

In [ ]:
avg = one[['SK_ID_CURR',"TARGET_AVG"]]

In [ ]:
avg.rename(columns={"TARGET_AVG":"TARGET"},inplace=True)

In [ ]:
avg.head()

,SK_ID_CURR,TARGET
0,7auxod,0.044368
1,xur7ur,0.202889
2,eau6rx,0.103046
3,obdr67,0.054001
4,7xbu7d,0.063881


In [ ]:
avg.to_csv("Elite_lgbm_8_9_10_avg.csv",index = False)